In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import pickle as pkl

import torch
from torch_geometric.data import Data

import wandb

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


True

In [2]:
# with open('datasets/normed_singletons_af_w_pza_graph_dict.pkl', 'rb') as f:
    # graph_dict = pkl.load(f)
with open('datasets/x19_exp_l2_graph_dict.pkl', 'rb') as f:
    graph_dict = pkl.load(f)
    
print(len(graph_dict['train']) + len(graph_dict['test']))

664


In [3]:
for t in graph_dict:
    for sample in graph_dict[t]:
        col_select = [i for i in range(19) if i != 14]
        new_feats = graph_dict[t][sample]['graph'].dataset[0].x[:, col_select]
        
        graph_dict[t][sample]['graph'].dataset[0].x = new_feats

### Parameters

Best param value ranges from WandB sweeps

- Cutoff distance = 11 - 12.5
- Dropout = 0.4-0.5
- Edge weights = "exp"
- Edge weight lambda = 2
- Hidden channels = 320
- Learning rate = 3.5e-5 - 4.5e-5
- Weight decay = 1e-6 - 1e-5

In [ ]:
seed = 42
np.random.seed(seed)
random.seed(seed)


# logging params (only used for wandb metrics)
n_samples = len(graph_dict['train']) + len(graph_dict['test'])
cutoff_distance = 12

# gcn params - from best wandb sweep
num_node_features = 18
batch_size = 256
hidden_channels = 256
dropout = 0.6

edge_weight_func = "exp"
edge_weight_lambda = 2

learning_rate = 4e-5
wd = 5e-6
epochs = 2000


wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [18]:
project = 'train-final-model'

In [21]:
# run_name = f'Run 7 - {epochs} epochs - lr {learning_rate}'
run_name = f'18 node feats - Run 5 - {epochs} epochs'

model = run_model.pnca_GCN_vary_graph(
            self_loops = False,
            cutoff_distance = cutoff_distance,
            edge_weight_func = edge_weight_func,
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = hidden_channels,
            learning_rate = learning_rate,
            wd = wd,
            dropout = dropout,
            lr_scheduling=False,
            epochs = epochs,
            graph_dict= graph_dict,
            normalise_ews=True,
            lambda_param= edge_weight_lambda,
            early_stop=False,
            # no_node_chem_feats=True,
            save_path= f'saved_models/{project}/{run_name}',
            wandb_params={
              'use_wandb': True, 
              'wandb_project': f'{project}', 
              'wandb_name': f'{run_name}',
              'n_samples': n_samples,
              'sweep': False
              }
        )

Using CUDA


Epoch: 000, Train Acc: 0.4935, Test Acc: 0.5200, Train Loss: 0.6933, Test Loss: 0.6931
Epoch: 010, Train Acc: 0.5259, Test Acc: 0.5150, Train Loss: 0.6913, Test Loss: 0.6920
Epoch: 020, Train Acc: 0.5323, Test Acc: 0.4800, Train Loss: 0.6899, Test Loss: 0.6907
Epoch: 030, Train Acc: 0.5409, Test Acc: 0.5200, Train Loss: 0.6876, Test Loss: 0.6898
Epoch: 040, Train Acc: 0.5280, Test Acc: 0.5150, Train Loss: 0.6864, Test Loss: 0.6895
Epoch: 050, Train Acc: 0.5259, Test Acc: 0.5200, Train Loss: 0.6849, Test Loss: 0.6879
Epoch: 060, Train Acc: 0.5647, Test Acc: 0.5200, Train Loss: 0.6840, Test Loss: 0.6862
Epoch: 070, Train Acc: 0.5474, Test Acc: 0.5100, Train Loss: 0.6815, Test Loss: 0.6850
Epoch: 080, Train Acc: 0.5797, Test Acc: 0.5150, Train Loss: 0.6793, Test Loss: 0.6827
Epoch: 090, Train Acc: 0.5884, Test Acc: 0.5300, Train Loss: 0.6766, Test Loss: 0.6800
Epoch: 100, Train Acc: 0.5560, Test Acc: 0.5150, Train Loss: 0.6722, Test Loss: 0.6779
Epoch: 110, Train Acc: 0.5776, Test Acc: 0.

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Test Accuracy,▁▁▂▅▅▆▅▇▆▆▇▆▇▇▇██████▆██▇███▆▆▇▇▇▇▇▅▇▇▆▆
Test F1,▂▂▂▃▁▂▁▅▄▄▇▃▆▅▆▇█▇▇▇█▆█▇▇▇▇▇▅▅▇▆▆▇▇▄▇▇▅▅
Test Loss,▅▅▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▂▂▂▄▄▂▃▃▃▃█▃▄▅▅
Test Sensitivity,██▇▃▁▂▁▃▃▂▅▂▄▂▃▄▅▄▄▄▅▅▅▄▅▄▄▄▅▅▄▅▃▄▅▆▅▅▅▅
Test Specificity,▁▁▂▇▇▇█▇▇█▇█▇██▇▇▇▇█▇▆▇█▇█▇▇▆▆▇▇█▇▇▄▇▇▆▆
Train Accuracy,▁▁▂▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▆▇▇▇▇██▆▇█▇▇██▆████
Train F1,▁▁▂▃▃▃▃▅▄▄▅▄▅▅▅▅▅▆▆▆▆▆▆▆▇▆▇▇▆▆█▇▇██▅████
Train Loss,███▇▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▃▂▂▂▂▁▁▃▁▁▁▁
Train Sensitivity,██▇▂▂▂▁▄▃▃▄▂▄▃▃▄▅▄▅▄▅▆▅▄▆▄▅▅▇▇▆▆▄▆▆▇▆▆▇▇
Train Specificity,▁▁▂▆▇▇▇▇▇▇▇█▇█▇▇▇▇▇█▇▇▇█▇███▆▇█▇███▅██▇▇
Test Accuracy,0.765


In [ ]:
# torch.save(model, f'saved_models/{project}/{run_name} - 0.80198 Test F1')

In [4]:
graph_dict['train']['pnca_mut_0']['graph'].dataset[0]

Data(x=[185, 18], edge_index=[2, 5452], edge_attr=[5452, 1], y=0)